<a href="https://colab.research.google.com/github/minit88/ML-Data_Analysis/blob/main/Price_Detection(cabbage)/Price_Detection(cabbage)(myself).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **기상데이터를 이용한 배추가격 예측**
# 직접만든 모델

In [63]:

import torch
import torch.nn as nn
import numpy as np
from pandas.io.parsers import read_csv
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
import torch.nn.functional as F
# csv 파일 불러오기 성공!
data = read_csv('/price data.csv',sep=',')
xy = np.array(data,dtype=np.float32)
print(data)

          year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0     20100101     -4.9    -11.0      0.9       0.0      2123
1     20100102     -3.1     -5.5      5.5       0.8      2123
2     20100103     -2.9     -6.9      1.4       0.0      2123
3     20100104     -1.8     -5.1      2.2       5.9      2020
4     20100105     -5.2     -8.7     -1.8       0.7      2060
...        ...      ...      ...      ...       ...       ...
2917  20171227     -3.9     -8.0      0.7       0.0      2865
2918  20171228     -1.5     -6.9      3.7       0.0      2884
2919  20171229      2.9     -2.1      8.0       0.0      2901
2920  20171230      2.9     -1.6      7.1       0.6      2901
2921  20171231      2.1     -2.0      5.8       0.4      2901

[2922 rows x 6 columns]


In [64]:
print(xy)


[[ 2.0100100e+07 -4.9000001e+00 -1.1000000e+01  8.9999998e-01
   0.0000000e+00  2.1230000e+03]
 [ 2.0100102e+07 -3.0999999e+00 -5.5000000e+00  5.5000000e+00
   8.0000001e-01  2.1230000e+03]
 [ 2.0100104e+07 -2.9000001e+00 -6.9000001e+00  1.4000000e+00
   0.0000000e+00  2.1230000e+03]
 ...
 [ 2.0171228e+07  2.9000001e+00 -2.0999999e+00  8.0000000e+00
   0.0000000e+00  2.9010000e+03]
 [ 2.0171230e+07  2.9000001e+00 -1.6000000e+00  7.0999999e+00
   6.0000002e-01  2.9010000e+03]
 [ 2.0171232e+07  2.0999999e+00 -2.0000000e+00  5.8000002e+00
   4.0000001e-01  2.9010000e+03]]


In [65]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [66]:

training_epochs = 1000


In [67]:
# 4개의 변인을 입력을 받습니다.
# avgTemp,minTemp,maxTemp,rainFall
x_data =torch.FloatTensor(xy[:,1:-1])


# 가격 값을 입력 받습니다.
# avgPrice
y_data =torch.FloatTensor(xy[:,[-1]])
dataset = TensorDataset(x_data,y_data)
data_loader = torch.utils.data.DataLoader(dataset,batch_size = 100,shuffle=True,drop_last=True)



In [68]:
# nn layers
linear1 = torch.nn.Linear(4,1,bias=True)

relu = torch.nn.ReLU()

# xavier initialization#
torch.nn.init.xavier_uniform_(linear1.weight)



Parameter containing:
tensor([[0.8349, 0.4913, 0.0072, 0.7117]], requires_grad=True)

In [69]:
# model 
model = torch.nn.Sequential(linear1,relu).to(device)

In [70]:
# define cost/loss & optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [71]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        #print(X,Y)
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        n=len(xy)
        hypothesis = model(X)
        #print(hypothesis)
        cost = F.mse_loss(hypothesis,Y)


        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 12414433.000000000
Epoch: 0002 cost = 12355708.000000000
Epoch: 0003 cost = 12230853.000000000
Epoch: 0004 cost = 12207908.000000000
Epoch: 0005 cost = 12090819.000000000
Epoch: 0006 cost = 11986678.000000000
Epoch: 0007 cost = 11948116.000000000
Epoch: 0008 cost = 11870693.000000000
Epoch: 0009 cost = 11750380.000000000
Epoch: 0010 cost = 11664821.000000000
Epoch: 0011 cost = 11566273.000000000
Epoch: 0012 cost = 11534164.000000000
Epoch: 0013 cost = 11423653.000000000
Epoch: 0014 cost = 11343114.000000000
Epoch: 0015 cost = 11288615.000000000
Epoch: 0016 cost = 11207285.000000000
Epoch: 0017 cost = 11127365.000000000
Epoch: 0018 cost = 11059355.000000000
Epoch: 0019 cost = 10943395.000000000
Epoch: 0020 cost = 10913034.000000000
Epoch: 0021 cost = 10826302.000000000
Epoch: 0022 cost = 10734345.000000000
Epoch: 0023 cost = 10693724.000000000
Epoch: 0024 cost = 10629390.000000000
Epoch: 0025 cost = 10554500.000000000
Epoch: 0026 cost = 10505093.000000000
Epoch: 0027 

In [73]:
# test file을 구하지 못해
# 학습한 W를 train data에 적용해봄

y_pred = model(x_data)
for i in range(1001):
  print('예측된 가격:',int(y_pred[i].item()))

예측된 가격: 1759
예측된 가격: 1925
예측된 가격: 1356
예측된 가격: 1452
예측된 가격: 976
예측된 가격: 1143
예측된 가격: 1358
예측된 가격: 1883
예측된 가격: 1679
예측된 가격: 1398
예측된 가격: 1280
예측된 가격: 925
예측된 가격: 748
예측된 가격: 1785
예측된 가격: 1951
예측된 가격: 1994
예측된 가격: 2368
예측된 가격: 2586
예측된 가격: 2964
예측된 가격: 2159
예측된 가격: 2286
예측된 가격: 1083
예측된 가격: 1432
예측된 가격: 2052
예측된 가격: 1830
예측된 가격: 2002
예측된 가격: 2597
예측된 가격: 1786
예측된 가격: 1742
예측된 가격: 2105
예측된 가격: 2211
예측된 가격: 1686
예측된 가격: 1266
예측된 가격: 1222
예측된 가격: 1703
예측된 가격: 1541
예측된 가격: 2172
예측된 가격: 2954
예측된 가격: 1411
예측된 가격: 1529
예측된 가격: 1286
예측된 가격: 1647
예측된 가격: 933
예측된 가격: 1132
예측된 가격: 1334
예측된 가격: 1739
예측된 가격: 1479
예측된 가격: 1703
예측된 가격: 1319
예측된 가격: 2035
예측된 가격: 2630
예측된 가격: 2906
예측된 가격: 2816
예측된 가격: 3551
예측된 가격: 3811
예측된 가격: 3126
예측된 가격: 2262
예측된 가격: 1625
예측된 가격: 2328
예측된 가격: 1826
예측된 가격: 1387
예측된 가격: 2348
예측된 가격: 1240
예측된 가격: 1580
예측된 가격: 1548
예측된 가격: 1355
예측된 가격: 1746
예측된 가격: 1633
예측된 가격: 1217
예측된 가격: 2230
예측된 가격: 2873
예측된 가격: 2466
예측된 가격: 2700
예측된 가격: 2965
예측된 가격: 1699
예측된 가격: 2033
예측된 가격: 1794
예측된